## Get the last several lines containing timing info

In [1]:
%%bash
mkdir -p timing
for x in 48 96 192 384 
do 
    for y in 20 40 80 160 
    do
        tail C${x}L${y}_std.log > timing/C${x}L${y}.log
    done
done

## Extract important numbers

In [2]:
import numpy as np
import pandas as pd

In [3]:
var_list = ['plume0{0}'.format(i) for i in range(1,5)]

Vres_list = [20,40,80,160]
Hres_list = [48,96,192,384]

Vlabels = ['L{0}'.format(r) for r in Vres_list]
Hlabels = ['C{0}'.format(r) for r in Hres_list]

In [4]:
def get_timing(logfile):
    with open(logfile, 'r') as fi:     
        for ln in fi:
            if 'PEs' in ln:
                CPU = ln.split()[-2]
            if 'Total runtime' in ln:
                time = ln.split()[4]

    return CPU, time

In [5]:
CPU_list = np.empty([4,4], dtype=int)
timing_list = np.empty([4,4])

for i, Hres in enumerate(Hres_list):
    for j, Vres in enumerate(Vres_list):
        logfile = './timing/'+'C{0}L{1}'.format(Hres,Vres)+'.log'
        CPU_list[i,j], timing_list[i,j] = get_timing(logfile)

In [6]:
df_CPU = pd.DataFrame(CPU_list, index=Hlabels, columns=Vlabels)
df_CPU

,L20,L40,L80,L160
C48,24,24,24,24
C96,24,24,24,24
C192,96,96,96,96
C384,384,384,384,384


In [7]:
df_time = pd.DataFrame(timing_list, index=Hlabels, columns=Vlabels)
df_time

,L20,L40,L80,L160
C48,30.624347,61.959384,125.105767,264.075614
C96,216.214940,434.180154,858.102888,1762.346824
C192,435.571059,863.830416,1770.668429,3591.502241
C384,834.573299,1726.190340,3576.954237,6798.907089


## Compute CPU x time and derivation from theory

In [8]:
df_CPUtime = df_time*df_CPU
df_CPUtime /= df_CPUtime.values.min() # normalize
df_CPUtime

,L20,L40,L80,L160
C48,1.000000,2.023207,4.085173,8.623061
C96,7.060230,14.177613,28.020284,57.547246
C192,56.892127,112.829236,231.275910,469.104173
C384,436.031266,901.865612,1868.815939,3552.157812


In [9]:
# ideal curve following dx^3 * dz=P

z_inc = 2**np.arange(4)
x_inc = 8**np.arange(4)

df_theory = pd.DataFrame(z_inc * x_inc.reshape(-1,1), index=Hlabels, columns=Vlabels)
df_theory 

,L20,L40,L80,L160
C48,1,2,4,8
C96,8,16,32,64
C192,64,128,256,512
C384,512,1024,2048,4096


In [10]:
# deviation from theory
df_CPUtime / df_theory 

,L20,L40,L80,L160
C48,1.000000,1.011603,1.021293,1.077883
C96,0.882529,0.886101,0.875634,0.899176
C192,0.888939,0.881478,0.903422,0.916219
C384,0.851624,0.880728,0.912508,0.867226


## Dump to files

In [11]:
df_CPU.to_csv('cpu.csv')
df_time.to_csv('time.csv')
df_CPUtime.to_csv('cpu_x_time.csv')
df_theory.to_csv('theory.csv')